In [262]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

titanic_df = pd.read_csv('data\\titanic\\train.csv')
titanic_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [232]:
titanic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [233]:
# 결측치 처리 Age, Cabin, Embarked

titanic_df['Age'].fillna(titanic_df['Age'].mean(), inplace=True)
titanic_df['Cabin'].fillna('N', inplace=True)
titanic_df['Embarked'].fillna('N', inplace=True)

C:\Users\LYJ\AppData\Local\Temp\ipykernel_12072\1152852216.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  titanic_df['Age'].fillna(titanic_df['Age'].mean(), inplace=True)
C:\Users\LYJ\AppData\Local\Temp\ipykernel_12072\1152852216.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a co

In [234]:
# titanic_df.info()

In [235]:
print(titanic_df['Age'].value_counts())
print(titanic_df['Cabin'].value_counts())
print(titanic_df['Embarked'].value_counts())

Age
29.699118    177
24.000000     30
22.000000     27
18.000000     26
28.000000     25
            ... 
24.500000      1
0.670000       1
0.420000       1
34.500000      1
74.000000      1
Name: count, Length: 89, dtype: int64
Cabin
N              687
G6               4
C23 C25 C27      4
B96 B98          4
F2               3
              ... 
E17              1
A24              1
C50              1
B42              1
C148             1
Name: count, Length: 148, dtype: int64
Embarked
S    644
C    168
Q     77
N      2
Name: count, dtype: int64


In [236]:
# drop PassengerId,Name,Sex
titanic_df = titanic_df.drop(['PassengerId','Name','Ticket'],axis=1)
titanic_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,male,22.0,1,0,7.2500,N,S
1,1,1,female,38.0,1,0,71.2833,C85,C
2,1,3,female,26.0,0,0,7.9250,N,S
3,1,1,female,35.0,1,0,53.1000,C123,S
4,0,3,male,35.0,0,0,8.0500,N,S


### label encoding -범주형

In [237]:
titanic_df['Cabin'] = titanic_df['Cabin'].str[:1]

from sklearn.preprocessing import LabelEncoder

col_names = ['Cabin','Sex','Embarked']

for col in col_names:
    le = LabelEncoder()
    le.fit(titanic_df[col])
    titanic_df[col] = le.transform(titanic_df[col])

titanic_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,1,22.0,1,0,7.2500,7,3
1,1,1,0,38.0,1,0,71.2833,2,0
2,1,3,0,26.0,0,0,7.9250,7,3
3,1,1,0,35.0,1,0,53.1000,2,3
4,0,3,1,35.0,0,0,8.0500,7,3


### Age.Fare - 수치형 그대로 or 스케일링 or 비닝

In [238]:
scaler = StandardScaler()
scaler.fit(titanic_df)
titanic_scaled = scaler.transform(titanic_df)
print(titanic_scaled.mean()) # 0에 가까운값

7.088591951715678e-18


In [239]:
titanic_scaled.var()

np.float64(1.0)

In [240]:
X_titanic_df = titanic_df.drop('Survived', axis=1)
y_titanic_df = titanic_df['Survived'] 

### 분할

In [242]:
X_train,X_test,y_train,y_test = train_test_split(X_titanic_df,y_titanic_df,test_size=0.2,random_state=11)

### 학습

In [243]:
dt_clf = DecisionTreeClassifier(random_state=11)
rf_clf = RandomForestClassifier(random_state=11)
lr_clf = LogisticRegression(solver='liblinear')

dt_clf.fit(X_train,y_train)
rf_clf.fit(X_train,y_train)
lr_clf.fit(X_train,y_train)

dt_pred = dt_clf.predict(X_test)
rf_pred = rf_clf.predict( X_test)
lr_pred = lr_clf.predict(X_test)

print(accuracy_score(y_test,dt_pred))
print(accuracy_score(y_test,rf_pred))
print(accuracy_score(y_test,lr_pred))


0.7877094972067039
0.8547486033519553
0.8659217877094972


### 교차 검증

kfold

In [244]:
# rf_clf_cv 평균 81.48%

def exe_kfold(clf, folds=5):

    kfold = KFold(folds) #stratified
    scores=[]

    for train_index, test_index in kfold.split(X_titanic_df) :
        X_train, X_test = X_titanic_df.values[train_index], X_titanic_df.values[test_index]
        y_train, y_test = y_titanic_df.values[train_index], y_titanic_df.values[test_index]

        clf.fit(X_train,y_train)
        pred = clf.predict(X_test)

        accuracy = accuracy_score(y_test,pred)
        scores.append(accuracy)
        
        print(accuracy)
    print(np.mean(scores))  

In [245]:
dt_clf_cv = DecisionTreeClassifier(random_state=11)
rf_clf_cv = RandomForestClassifier(random_state=11)
lr_clf_cv = LogisticRegression(solver='liblinear')

print(exe_kfold(dt_clf_cv, folds=5))
print(exe_kfold(rf_clf_cv, folds=5))
print(exe_kfold(lr_clf_cv, folds=5))


0.7541899441340782
0.7808988764044944
0.7865168539325843
0.7696629213483146
0.8202247191011236
0.782298662984119
None
0.7932960893854749
0.8089887640449438
0.8370786516853933
0.7752808988764045
0.8595505617977528
0.8148389931579938
None
0.7932960893854749
0.7921348314606742
0.7752808988764045
0.7471910112359551
0.8426966292134831
0.7901198920343984
None


cross_val_score

In [246]:
# cross_val_score - rf_clf_gcv 제일 높게 나옴 평균 81.37%

def cvs(clf):
    scores = cross_val_score(clf, X_titanic_df, y_titanic_df, cv=5)

    print(scores)
    print(np.mean(scores))

In [247]:
dt_clf_gcv = DecisionTreeClassifier(random_state=11)
rf_clf_gcv = RandomForestClassifier(random_state=11)
lr_clf_gcv = LogisticRegression(solver='liblinear')

print(cvs(dt_clf_gcv))
print(cvs(rf_clf_gcv))
print(cvs(lr_clf_gcv))

[0.74301676 0.7752809  0.79213483 0.78651685 0.84269663]
0.7879291946519366
None
[0.79329609 0.79775281 0.84831461 0.76404494 0.86516854]
0.8137153976523758
None
[0.7877095  0.79213483 0.7752809  0.76404494 0.81460674]
0.7867553825874082
None


GridSearchCV 파라미터 튜닝

In [248]:
# DecisionTree parameter
parameters = { 'max_depth': [2,3,5,10], 'min_samples_split': [1,5,8]}

grid_dt_clf = GridSearchCV(dt_clf_gcv, param_grid=parameters, scoring='accuracy', cv=5,refit=True)

grid_dt_clf.fit(X_train, y_train)

scored_df = pd.DataFrame(grid_dt_clf.cv_results_)
scored_df[:3]

c:\Users\LYJ\anaconda3\envs\ml_env\lib\site-packages\sklearn\model_selection\_validation.py:528: FitFailedWarning: 
20 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\LYJ\anaconda3\envs\ml_env\lib\site-packages\sklearn\model_selection\_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\LYJ\anaconda3\envs\ml_env\lib\site-packages\sklearn\base.py", line 1382, in wrapper
    estimator._validate_params()
  File "c:\Users\LYJ\anaconda3\envs\ml_env\lib\site-packages\sklearn\base.py", line 436, in _validate_params
    validate_parameter_constraints(
  File "c:\

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_min_samples_split,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001200,0.000400,0.0000,0.000000,2,1,"{'max_depth': 2, 'min_samples_split': 1}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
1,0.004401,0.001855,0.0018,0.000400,2,5,"{'max_depth': 2, 'min_samples_split': 5}",0.734266,0.769231,0.753521,0.788732,0.746479,0.758446,0.018898,7
2,0.003401,0.001855,0.0018,0.000399,2,8,"{'max_depth': 2, 'min_samples_split': 8}",0.734266,0.769231,0.753521,0.788732,0.746479,0.758446,0.018898,7


In [263]:
# RandomForest parameter
parameters = { 'max_depth': [2,3,5,10], 'min_samples_split': [1,5,8]}

grid_rf_clf = GridSearchCV(rf_clf_gcv, param_grid=parameters, scoring='accuracy', cv=5,refit=True)

grid_rf_clf.fit(X_train, y_train)

scored_df = pd.DataFrame(grid_rf_clf.cv_results_)
scored_df[:3]

c:\Users\LYJ\anaconda3\envs\ml_env\lib\site-packages\sklearn\model_selection\_validation.py:528: FitFailedWarning: 
20 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\LYJ\anaconda3\envs\ml_env\lib\site-packages\sklearn\model_selection\_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\LYJ\anaconda3\envs\ml_env\lib\site-packages\sklearn\base.py", line 1382, in wrapper
    estimator._validate_params()
  File "c:\Users\LYJ\anaconda3\envs\ml_env\lib\site-packages\sklearn\base.py", line 436, in _validate_params
    validate_parameter_constraints(
  File "c:\

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_min_samples_split,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.002199,0.000982,0.000000,0.000000,2,1,"{'max_depth': 2, 'min_samples_split': 1}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
1,0.216304,0.026973,0.017404,0.006719,2,5,"{'max_depth': 2, 'min_samples_split': 5}",0.762238,0.804196,0.774648,0.78169,0.746479,0.77385,0.019324,7
2,0.300799,0.069042,0.024200,0.007139,2,8,"{'max_depth': 2, 'min_samples_split': 8}",0.762238,0.804196,0.774648,0.78169,0.746479,0.77385,0.019324,7


In [260]:
# 최적의 모델
best_dt_clf = grid_dt_clf.best_estimator_
best_rf_clf = grid_rf_clf.best_estimator_

#예측
best_dt_pred = best_dt_clf.predict(X_test)
best_rf_pred = best_rf_clf.predict(X_test)

print(accuracy_score(y_test, best_dt_pred))
print(accuracy_score(y_test, best_rf_pred))

0.8715083798882681
0.88268156424581


In [256]:
print(grid_dt_clf.best_params_, grid_dt_clf.best_score_)
print(grid_rf_clf.best_params_, grid_rf_clf.best_score_)

{'max_depth': 3, 'min_samples_split': 5} 0.799172658327588
{'max_depth': 10, 'min_samples_split': 8} 0.8174332709543977


In [252]:
# 평가

# 정확도 높이기 - 튜닝
- 스케일링
- 속성선택
- model : RF, LR
- 파라미터